In [6]:
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
# from scipy.sparse import csr_matrix
# import faiss
import random

warnings.filterwarnings(action='ignore')
# books = pd.read_csv('/content/drive/MyDrive/nlbooks.csv', encoding='utf-8')
# books = books.replace({np.nan: 'none'})
books = pd.read_csv('/content/drive/MyDrive/books_mod2', encoding='utf-8')
books = books.replace({np.nan: 'none'})

# 데이터프레임이 content, thumbnail column을 포함한다면
# books_df=books[['isbn13','title','author','publisher','price','img_url','description','kdc_class_no']]
# books_df=books[['isbn13','title','author','publisher','price','img_url','description','kdc_class_no']]

#books_df[['description']=='none']=None
books_df_nax=books.dropna(axis='rows')


tfidf = TfidfVectorizer(stop_words='english')
# fit_transform안에 데이터프레임형태로 넣어주면 안됨. 하나의 변수씩만 넣어주자!
# title_vect1 = cnt_vect.fit_transform(books_df_nax['title'])
# title_vect2 = tfidf.fit_transform(books_df_nax['title'])
#des_vect = cnt_vect.fit_transform(books['description'])
#books['book_info']=books['title']+books['description']
#b_category=books['kdc_class_no']

# 차원 축소 part 를 줄여야 함 -> 파일을 새로 저장해야할듯!
#X = title_vect2
#svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
#X_new=svd.fit_transform(X)

#D = X_new.shape[1]
#K = 500  # The number of clusters
#X = X_new.astype(np.float32)

# Setup
#kmeans = faiss.Kmeans(d=D, k=K, niter=20, verbose=True,gpu=True)
# For GPU(s), run the following line. This will use all GPUs
# kmeans = faiss.Kmeans(d=D, k=K, niter=20, verbose=True, gpu=True)

# Run clustering
#kmeans.train(X)

# Error for each iteration
# print(kmeans.obj)  # array with 20 elements

# Centroids after clustering
# print(kmeans.centroids.shape)  # (10, 500)

# The assignment for each vector.
#dists, ids = kmeans.index.search(X, 1)  # Need to run NN search again


#books_df_nax['tit_cluster']=ids

#books_df_nax

def find_sim_books(U):
    def userbooks(A):
        l=len(A)
        n=random.randrange(0,l)
        return A[n]

    # 사용자의 isbn 데이터를 담은 책 리스트를 input으로 받아 그 리스트 중 하나를 반환하는 함수
    # B = 사용자 데이터에서 랜덤 추출한 책의 isbn 
    B=userbooks(U) 


    def sim_idx_with_A(df,isbn):
        n=int(df[df['isbn']==isbn]['tit_cluster'])
        same_clu_books_df=df[df['tit_cluster']==n]
        sen = same_clu_books_df['content']

        # 1번 후보 : sentence transformer 유사도 구하기
        #from sentence_transformers import SentenceTransformer
        #model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        #sen_embeddings = model.encode(sen)
    
        # 2번 후보 : 단순 tfidf 활용한 유사도 구하기
        clu_tfidf = tfidf.fit_transform(sen)
        clu_des_sim=cosine_similarity(clu_tfidf,clu_tfidf)

        # idx_list=same_clu_books_df.index[same_clu_books_df['title']==books].tolist()
        idx=np.where(same_clu_books_df['isbn']==B)[0][0]
        books_sim_vect=clu_des_sim[idx:idx+1]
        books_des_sim_idx = books_sim_vect.argsort()[::-1]

        return books_des_sim_idx

    # 사용자 책 리스트에서 랜덤 추출한 책 기준 유사한 책 인덱스 리스트 반환
    sim_books_idx=sim_idx_with_A(books_df_nax,B)

    #books=userbooks(A)
    #n=int(df[df['title']==books]['tit_cluster'])
    #same_clu_books_df=df[df['tit_cluster']==n]
    #sen = same_clu_books_df['description']
    top_n=20
    top_sim_idx=sim_books_idx[0][:top_n]
    top_sim_idx=top_sim_idx.reshape(-1,)

    sim_books=books_df_nax.iloc[top_sim_idx]
    # 데이터프레임이 content, thumbnail column을 포함한다면
    outputs=sim_books[['title','author','content','thumbnail','publisher','kdc','price']]
    # outputs=sim_books[['title','author','img_url','publisher','kdc_class_no','price']]
    outputs_dic=outputs.to_dict('records')

    return outputs_dic

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
find_sim_books([9791156759270])

[{'title': '그 너머의 시 - 오하룡 시집',
  'author': '오하룡 (지은이)',
  'content': '오하룡 시집 『그 너머의 시』는 『시집 밖의 시』를 펴낸 지 4년 만에 나왔다. 이번 시집에서 보여주는 그의 시력과 시안(詩眼)은 원숙하면서, 대상에 대한 질문과 그것을 끌어안는 서로 간의 교통은 불교의 원융무애와 연대하고 있다.',
  'thumbnail': 'https://image.aladin.co.kr/product/28383/37/cover/k652835711_1.jpg',
  'publisher': '황금알',
  'kdc': '811.7',
  'price': '10000.0'},
 {'title': 'Plinky Witch and the Grand Halloween Scheme: A Funny Halloween Tale for Kids Ages 4-8 (Paperback)',
  'author': 'Liz Cooper (지은이)',
  'content': 'Plinky Witch, a clever little witch wearing striped stockings, has a thought one day-Halloween would beso much more fun if it occurred every single night, instead of only once a year! The members of theHappy Broomsticks Club agree. They work hard to create a &quot;forever&quot; trick...',
  'thumbnail': 'https://image.aladin.co.kr/product/28083/63/cover/1956783008_2.jpg',
  'publisher': 'Elizabeth Cooper',
  'kdc': '843',
  'price': '14980.0'},
 {'title': 'I love you more than worms love 